In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops
ops.reset_default_graph()
import os

In [29]:
sess = tf.Session()

# 1 数值之间的运算

## 1.2 除法运算
`div()` : 整除

`truediv()` : 整数与整数相除，得到的结果为浮点型

`floordiv()` : float of `div()` 浮点数据与浮点数据相除

In [4]:
print(sess.run(tf.div(3,4)))
print(sess.run(tf.truediv(3,4)))
print(sess.run(tf.floordiv(3.0,4.0)))

0
0.75
0.0


## 1.2 取余数

In [5]:
print(sess.run(tf.mod(22.0,5.0)))

2.0


## 1.3 负负得正的运算

In [6]:
print(sess.run(tf.cross([1.,0.,0.],[0.,1.,0.])))

[ 0.  0.  1.]


## 1.4 三角函数运算

In [7]:
print(sess.run(tf.sin(3.1416)))
print(sess.run(tf.cos(3.1416)))
print(sess.run(tf.div(tf.sin(3.1416/4.), tf.cos(3.1416/4.))))

-7.23998e-06
-1.0
1.0


## 1.5 创建运算符号

`f(x) = 3 * x^2 - x + 10`

tf.mul  tf.sub   tf.neg 已经废弃 

分别可用tf.multiply减法  tf.subtract加法  tf.negative取相反数 替代.

In [8]:
test_nums = range(15)

def custom_polynomial(x_val):
    # Return 3x^2 - x + 10
    return(tf.subtract(3 * tf.square(x_val), x_val) + 10)

print(sess.run(custom_polynomial(11)))

362


In [9]:
expected_output = [3*x*x-x+10 for x in test_nums]
print(expected_output)

[10, 12, 20, 34, 54, 80, 112, 150, 194, 244, 300, 362, 430, 504, 584]


In [10]:
for num in test_nums:
    print(sess.run(custom_polynomial(num)))

10
12
20
34
54
80
112
150
194
244
300
362
430
504
584


# 2 单层运算
把tensor constants placeholders创建的变量当做层，像四则运算一样融合这些层


In [11]:
# Create data to feed in
my_array = np.array([[1., 3., 5., 7., 9.],
                   [-2., 0., 2., 4., 6.],
                   [-6., -3., 0., 3., 6.]])
# Duplicate the array for having two inputs
x_vals = np.array([my_array, my_array + 1])
# Declare the placeholder
x_data = tf.placeholder(tf.float32, shape=(3, 5))
# Declare constants for operations
m1 = tf.constant([[1.],[0.],[-1.],[2.],[4.]])
m2 = tf.constant([[2.]])
a1 = tf.constant([[10.]])

## 2.1 层的乘法

A[3x5] * m1[5x1] = prod1[3x1]

In [12]:
prod1 = tf.matmul(x_data, m1)

In [13]:
prod2 = tf.matmul(prod1, m2)

## 2.2 层的加法

In [14]:
add1 = tf.add(prod2, a1)

## 2.3 层之间运算的输出

In [15]:
for x_val in x_vals:
    print(sess.run(add1, feed_dict={x_data: x_val}))

[[ 102.]
 [  66.]
 [  58.]]
[[ 114.]
 [  78.]
 [  70.]]


In [16]:
merged = tf.summary.merge_all(key='summaries')

if not os.path.exists('/tmptensorboard_logs/'):
    os.makedirs('/tmp/tensorboard_logs/')

my_writer = tf.summary.FileWriter('/tmp/tensorboard_logs/', sess.graph)

![layering_nested_operations](https://github.com/nfmcclure/tensorflow_cookbook/raw/master/02_TensorFlow_Way/images/02_Multiple_Operations.png)

# 3 多层运算

### 3.0 创建数据和数据对应的占位符

In [30]:
x_val = np.random.uniform(size=[1, 4, 4, 1])
x_data = tf.placeholder(tf.float32, shape=[1, 4, 4, 1])
print(x_val)

[[[[ 0.55584819]
   [ 0.3433674 ]
   [ 0.77458665]
   [ 0.64840371]]

  [[ 0.56442104]
   [ 0.25010278]
   [ 0.90336648]
   [ 0.19636605]]

  [[ 0.30169955]
   [ 0.73136795]
   [ 0.84871716]
   [ 0.73755175]]

  [[ 0.52037057]
   [ 0.64754771]
   [ 0.3777954 ]
   [ 0.38315488]]]]


## 3.1 第一层的卷积运算

In [31]:
my_filter = tf.constant(0.25, shape=[2, 2, 1, 1])  # 卷积核
my_strides = [1, 2, 2, 1]
mov_avg_layer= tf.nn.conv2d(x_data, my_filter, my_strides, padding='SAME', name='Moving_Avg_Window')

In [34]:
# 输出卷积层的结果
print(sess.run(mov_avg_layer, feed_dict={x_data: x_val}))

[[[[ 0.42843485]
   [ 0.63068068]]

  [[ 0.55024642]
   [ 0.58680481]]]]


## 3.2 多层之间的运算

### 3.2.1 tf.expand_dims(input, axis=None, name=None, dim=None)
在轴上添加1的一个维度

1、 t is a tensor of shape [2]

shape(expand_dims(t, 0)) ==> [1, 2]

shape(expand_dims(t, 1)) ==> [2, 1]

shape(expand_dims(t, -1)) ==> [2, 1]


2、 t is a tensor of shape [2, 3, 5]

shape(expand_dims(t2, 0)) ==> [1, 2, 3, 5]

shape(expand_dims(t2, 2)) ==> [2, 3, 1, 5]

shape(expand_dims(t2, 3)) ==> [2, 3, 5, 1]

### 3.2.2 tf.squeeze(input, squeeze_dims=None, name=None)
删除所有为1的维度

1、 't' is a tensor of shape [1, 2, 1, 3, 1, 1]
shape(squeeze(t)) ==> [2, 3]

Or, to remove specific size 1 dimensions:

2、 't' is a tensor of shape [1, 2, 1, 3, 1, 1]

shape(squeeze(t, [2, 4])) ==> [1, 2, 3, 1]

In [32]:
def custom_layer(input_matrix):
    input_matrix_sqeezed = tf.squeeze(input_matrix)
    A = tf.constant([[1., 2.], [-1., 3.]])
    b = tf.constant(1., shape=[2, 2])
    temp1 = tf.matmul(A, input_matrix_sqeezed)
    temp = tf.add(temp1, b) # Ax + b
    return(tf.sigmoid(temp))

# Add custom layer to graph
with tf.name_scope('Custom_Layer') as scope:
    custom_layer1 = custom_layer(mov_avg_layer)

In [33]:
print(sess.run(custom_layer1, feed_dict={x_data: x_val}))

[[ 0.92614508  0.94290721]
 [ 0.90223467  0.89375973]]


In [35]:
merged = tf.summary.merge_all(key='summaries')

if not os.path.exists('/tmp/tensorboard_logs/'):
    os.makedirs('/tmp/tensorboard_logs/')

my_writer = tf.summary.FileWriter('/tmp/tensorboard_logs/', sess.graph)

![multiple_layers_tensorboard](https://github.com/nfmcclure/tensorflow_cookbook/raw/master/02_TensorFlow_Way/images/03_Multiple_Layers.png)

In [28]:
sess.close()